## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [2]:
!pip install python-dotenv


  Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [7]:
!pip install langchain-groq


  Using cached langchain_groq-0.3.2-py3-none-any.whl (15 kB)
  Using cached groq-0.28.0-py3-none-any.whl (130 kB)
  Using cached langchain_core-0.3.65-py3-none-any.whl (438 kB)
  Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
  Using cached anyio-4.9.0-py3-none-any.whl (100 kB)
  Using cached distro-1.9.0-py3-none-any.whl (20 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl (444 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl (28 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
  Using cached langsmith-0.3.45-py3-none-any.whl (363 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl (161 kB)
  Using cached packaging-24.2-py3-none-any.whl (65 kB)
  Using cached idna-3.10-py3-none-any.whl (70 kB)
     -------------------------------------- 157.7/157.7 kB 1.9 MB/s eta 0:00:00
  Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
  Using cached h11-0.16.0-py3-none-any.whl (37 kB)
  Using cached jsonpointer-


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B0CEE06BC0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B0CEE04670>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Smriti and I am a Chief AI Engineer")])

AIMessage(content="Hello Smriti, it's a pleasure to meet you!\n\nBeing a Chief AI Engineer is a fascinating role. What kind of projects are you currently working on?  \n\nI'm always eager to learn more about the advancements happening in the field of AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 23, 'total_tokens': 80, 'completion_time': 0.103636364, 'prompt_time': 0.002113427, 'queue_time': 0.249860203, 'total_time': 0.105749791}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--d7325339-aee2-4a48-b94f-664530d6f430-0', usage_metadata={'input_tokens': 23, 'output_tokens': 57, 'total_tokens': 80})

In [10]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Smriti and I am a Chief AI Engineer"),
        AIMessage(content="Hello Smriti! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Smriti, and you are a Chief AI Engineer!  😊 \n\nIs there anything else you'd like to tell me about yourself or your work? I'm happy to chat. \n\n\n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 99, 'total_tokens': 144, 'completion_time': 0.081818182, 'prompt_time': 0.004365863, 'queue_time': 0.25256314799999996, 'total_time': 0.086184045}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--15287748-7936-482a-b4f5-51de3f3c9b00-0', usage_metadata={'input_tokens': 99, 'output_tokens': 45, 'total_tokens': 144})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [11]:
!pip install langchain_community

  Using cached langchain_community-0.3.25-py3-none-any.whl (2.5 MB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
  Using cached pydantic_settings-2.9.1-py3-none-any.whl (44 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl (12.9 MB)
  Using cached langchain-0.3.25-py3-none-any.whl (1.0 MB)
  Using cached aiohttp-3.12.13-cp310-cp310-win_amd64.whl (450 kB)
  Using cached sqlalchemy-2.0.41-cp310-cp310-win_amd64.whl (2.1 MB)
  Using cached yarl-1.20.1-cp310-cp310-win_amd64.whl (86 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
  Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl (7.6 kB)
  Using cached frozenlist-1.7.0-cp310-cp310-win_amd64.whl (43 kB)
  Using cached multidict-6.4.4-cp310-cp310-win_amd64.whl (38 kB)
  Using cached propcache-0.3.2-cp310-cp310-win_amd64.whl (41 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 k


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [13]:
config={"configurable":{"session_id":"chat1"}}

In [16]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Smriti and I am a Chief AI Engineer")],
    config=config
)

In [17]:
response.content

"Hi Smriti!  It's great to meet you. \n\nBeing a Chief AI Engineer is a fascinating role. What kind of AI projects are you leading these days?  I'm always interested in hearing about the innovative work being done in the field.  \n\n"

In [18]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Smriti.  😊 \n\nI remembered from our earlier conversation. \n\n\n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 166, 'total_tokens': 198, 'completion_time': 0.058181818, 'prompt_time': 0.007017899, 'queue_time': 0.25173724000000003, 'total_time': 0.065199717}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--2b38e866-0661-402a-9d41-37449582b52e-0', usage_metadata={'input_tokens': 166, 'output_tokens': 32, 'total_tokens': 198})

In [19]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and don't know your name. If you'd like to tell me, I'm happy to know! 😊\n"

In [20]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

'Nice to meet you, John! 👋  Is there anything I can help you with today? \n'

In [21]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John,  remember? 😊  How can I help you today?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [22]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [23]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Smriti")]})

AIMessage(content="Hi Smriti, it's nice to meet you!\n\nI'm happy to help with any questions you have. Just ask away!  😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 32, 'total_tokens': 67, 'completion_time': 0.063636364, 'prompt_time': 0.002342206, 'queue_time': 0.253632804, 'total_time': 0.06597857}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--7e058363-1f2c-4082-8bec-e5d9645f2085-0', usage_metadata={'input_tokens': 32, 'output_tokens': 35, 'total_tokens': 67})

In [24]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [25]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Smriti")],
    config=config
)

response

AIMessage(content="Hi Smriti, it's nice to meet you! \n\nI'm happy to help with any questions you have. Just ask away, and I'll do my best to provide you with useful and informative answers. 😊 \n\nWhat can I do for you today?  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 32, 'total_tokens': 94, 'completion_time': 0.112727273, 'prompt_time': 0.002358676, 'queue_time': 0.25102839, 'total_time': 0.115085949}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--eab43e13-0c88-4bec-bd84-999dd6e69d9a-0', usage_metadata={'input_tokens': 32, 'output_tokens': 62, 'total_tokens': 94})

In [26]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Smriti.  I remember! 😊  \n\nHow can I help you further? \n\n\n'

In [27]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [29]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Smriti")],"language":"Hindi"})
response.content

'नमस्ते स्मृति! 👋 \n\nमुझे बहुत खुशी है कि आप मुझसे बात करना चाहती हैं। 😊 मैं आपकी मदद करने के लिए तैयार हूँ।  \n\nआप किस बारे में पूछना चाहती हैं?  \n\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [30]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [31]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Smriti")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते Smriti! मुझे बताओ, मैं तुम्हारी कैसे मदद कर सकता हूँ? 😊 \n'

In [32]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [33]:
response.content

'तुम्हारा नाम Smriti है! 😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [36]:
!pip install transformers

  Using cached transformers-4.52.4-py3-none-any.whl (10.5 MB)
  Using cached huggingface_hub-0.33.0-py3-none-any.whl (514 kB)
  Using cached regex-2024.11.6-cp310-cp310-win_amd64.whl (274 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl (308 kB)
  Using cached filelock-3.18.0-py3-none-any.whl (16 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl (2.4 MB)
  Using cached fsspec-2025.5.1-py3-none-any.whl (199 kB)



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

c:\Users\KIIT0001\OneDrive\Desktop\ChatbotGenAI\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
c:\Users\KIIT0001\OneDrive\Desktop\ChatbotGenAI\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\KIIT0001\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [38]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a large language model, I don't have access to your personal information, including your favorite ice cream flavor! \n\nWhat's your favorite ice cream? 😊🍦\n"

In [39]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked: 2 + 2'

In [40]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [41]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name.  If you'd like to tell me your name, I'd be happy to know! 😊  \n"

In [42]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a helpful assistant, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just ask away! \n\n"